# Foo
This is just a temporary solution, this needs to be a part of longipy, not photostim_deve...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get five random numbers between 0 and 1312
np.random.seed(42)
rand_nrs = np.random.randint(0, 500, size=15)
print(rand_nrs)

In [ ]:
data_dir = 'data_proc' # data_loc is  the directory on local ssd (only two sessions, one for jm049 and one for jm048)
experimenter = 'jm'
mouse = 'jm064' # 'jm049' or 'jm048'
session =  '2025-11-13_s' # '2025-05-23_b' or '2025-05-08_c'

stim_wind = (10, 120)

# plot_nrns = [7, 15, 226, 116, 67, 0, 1, 2, 3, 4, 684, 559, 1216, 835, 763] # jm055
# plot_nrns = [0, 1, 14, 35, 51, 4, 8, 22, 29, 2, 684, 559, 629, 192, 835] # jm056
# get 15 random numbers between 0 and 500
plot_nrns = rand_nrs.tolist()
# labels = ['resp', 'resp', 'resp', 'resp', 'resp', 'state', 'state', 'state', 'state', 'state', 'rand', 'rand', 'rand', 'rand', 'rand']
labels = ['rand'] * 15

In [ ]:
F_allroi = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/suite2p/plane0/F.npy')
stim_protocol = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/stim_protocol.npy')
stim_times = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/stim_times.npy')

In [ ]:
# filter by iscell and plot only cells
iscell = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/suite2p/plane0/iscell.npy')
cell_indices = np.where(iscell[:, 0] == 1)[0]
F = F_allroi[cell_indices, :]
print(F.shape)  # (num_cells, num_timepoints)

In [ ]:
# now zscore rows, run pca and then plot scatter of first two pcs
from sklearn.decomposition import PCA

F_zscored = (F - np.mean(F, axis=1, keepdims=True)) / np.std(F, axis=1, keepdims=True)
# remove nan rows
valid_rows = ~np.isnan(F_zscored).any(axis=1)
F_zscored_valid = F_zscored[valid_rows]

pca = PCA(n_components=2)
F_pca = pca.fit_transform(F_zscored_valid.T)
print(F_pca.shape)  # (num_cells, 2)
# plot
plt.figure(figsize=(8, 6))
plt.scatter(F_pca[2700:3900, 0], F_pca[2700:3900, 1], c=np.arange(1200), cmap='jet', s=5)
plt.colorbar(label='time')

In [ ]:
for i in plot_nrns:
    plt.figure(figsize=(20, 2))
    plt.plot(F[i])
    for st in stim_times:
        plt.axvline(st, color='r')

In [ ]:
stim_array = np.zeros((len(plot_nrns), len(stim_times), stim_wind[0] + stim_wind[1]))
for i in range(len(plot_nrns)):
    for j, st in enumerate(stim_times):
        stim_array[i, j, :] = F[plot_nrns[i], st - stim_wind[0]:st + stim_wind[1]]
        # subtract baseline
        stim_array[i, j, :] = stim_array[i, j, :] - np.mean(stim_array[i, j, :stim_wind[0]])
        

In [ ]:
# now plot the heatmap for each neuron
fig, axs = plt.subplots(3, 5, figsize=(10, 5), dpi=300)
for i in range(len(plot_nrns)):
    ax = axs[i // 5, i % 5]
    plt.sca(ax)
    plt.imshow(stim_array[i, :, :], aspect='auto', cmap='bwr', vmin = -5*np.std(stim_array[i, :, :]), vmax = 5*np.std(stim_array[i, :, :]))
    # if first column, add ylabel
    # remove xticks and yticks
    plt.xticks([])
    plt.yticks([])
    if i % 5 == 0:
        plt.ylabel(labels[i])
        # add yticks
        plt.yticks([0, stim_array.shape[1]//2, stim_array.shape[1]-1], [0, stim_array.shape[1]//2, stim_array.shape[1]-1])
    # if last row, add xlabel
    if i // 5 == 2:
        plt.xlabel('Time (frames)')
        plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])
    
    plt.suptitle(f'Responses to evoked stim for 5 resp, 5 state and 5 rand neurons')
    


In [ ]:
# plot the average +-std for resp neurons
fig, axs = plt.subplots(1, 5, figsize=(10, 1.5), dpi=300)

for i in range(5):
    ax = axs[i]
    plt.sca(ax)
    mean_trace = np.mean(stim_array[i, :, :], axis=0)
    std_trace = np.std(stim_array[i, :, :], axis=0)
    plt.plot(mean_trace, color=f'C{i}')
    plt.fill_between(np.arange(len(mean_trace)), mean_trace - std_trace, mean_trace + std_trace, color=f'C{i}', alpha=0.1)
    plt.title(f'Resp neuron {plot_nrns[i]}')
    plt.axvline(stim_wind[0], color='grey', linestyle='--')
    plt.xlabel('Time (frames)')
    if i == 0:
        plt.ylabel('dF/F')
    plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])
    # keep y ticks but remove labels
    ax.set_yticklabels([])
    # remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)


In [ ]:
# now show all neurons together
fig, axs = plt.subplots(3, 5, figsize=(10, 5), dpi=300)
for i in range(len(plot_nrns)):
    ax = axs[i // 5, i % 5]
    plt.sca(ax)
    mean_trace = np.mean(stim_array[i, :, :], axis=0)
    std_trace = np.std(stim_array[i, :, :], axis=0)
    plt.plot(mean_trace, color=f'C{i}')
    plt.fill_between(np.arange(len(mean_trace)), mean_trace - std_trace, mean_trace + std_trace, color=f'C{i}', alpha=0.1)
    plt.axvline(stim_wind[0], color='grey', linestyle='--')
    if i % 5 == 0:
        plt.ylabel('dF/F')
    plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])

    if i // 5 == 2:
        plt.xlabel('Time (frames)')
    # keep y ticks but remove labels
    ax.set_yticklabels([])
    # remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)   